In [34]:
# Import necessary libraries
import ephem
import math
from datetime import timedelta
from datetime import datetime


# get_passes() function definition
def get_passes(observer, tle, start_time, num_passes=None, duration=None):
    """Config obs and sat, Return pass data for all passes in given interval.

    Arguments:
    observer -- 4 element list containing desired [name,lat,lon,alt]
    tle -- 3 element list containing desired tle [line0,line1,line2]
    start_time -- ephem.date string formatted 'yyyy/mm/dd hr:min:sec'
    num_passes -- integer number of desired passes (defualt None)
    duration -- float number of hours or fraction of hours (default None)

    Specify either num_passes or duration.
    If both, use num_passes.
    If neither, find passes for next 24 hours.
    """

    obs_name, obs_lat, obs_lon, obs_alt = observer
    tle_line0, tle_line1, tle_line2 = tle

    # Set up location of observer
    ground_station = ephem.Observer()
    ground_station.name = obs_name                # name string
    ground_station.lon = obs_lon                  # in degrees (+E)
    ground_station.lat = obs_lat                  # in degrees (+N)
    ground_station.elevation = obs_alt            # in meters
    ground_station.date = ephem.date(start_time)  # in UTC

    # Read in most recent satellite TLE data
    sat = ephem.readtle(tle_line0, tle_line1, tle_line2)

    contacts = []

    if num_passes is not None and duration is None:
        # if only num_passes specified
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(days = set_time-rise_time)  # fraction of a day

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time
                    contacts.append(pass_data)

                # increase by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute

        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is None and duration is not None:
        # if only duration specified
        try:
            end_time = ephem.date(ground_station.date+duration*ephem.hour)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time
                    contacts.append(pass_data)

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute

        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is not None and duration is not None:
        # if both are specified, use num_passes
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time
                    contacts.append(pass_data)

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute

        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is None and duration is None:
        # if neither are specified, get passes for the next 24 hours
        try:
            end_time = ephem.date(ground_station.date+1)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time
                    contacts.append(pass_data)

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute

        except ValueError:
            # No (more) visible passes
            pass
        return contacts


In [48]:
# Import necessary libraries
import ephem
import math
from datetime import timedelta
from datetime import datetime


# get_passes() function definition
def new_get_passes(observer, tle, start_time, num_passes=None, duration=None):
    """Config obs and sat, Return pass data for all passes in given interval.

    Arguments:
    observer -- 4 element list containing desired [name,lat,lon,alt]
    tle -- 3 element list containing desired tle [line0,line1,line2]
    start_time -- ephem.date string formatted 'yyyy/mm/dd hr:min:sec'
    num_passes -- integer number of desired passes (defualt None)
    duration -- float number of hours or fraction of hours (default None)

    Specify either num_passes or duration.
    If both, use min(num_passes, duration).
    If neither, find passes for next 24 hours.
    """

    obs_name, obs_lat, obs_lon, obs_alt = observer
    tle_line0, tle_line1, tle_line2 = tle

    # Set up location of observer
    ground_station = ephem.Observer()
    ground_station.name = obs_name                # name string
    ground_station.lon = obs_lon                  # in degrees (+E)
    ground_station.lat = obs_lat                  # in degrees (+N)
    ground_station.elevation = obs_alt            # in meters
    ground_station.date = ephem.date(start_time)  # in UTC

    # Read in most recent satellite TLE data
    sat = ephem.readtle(tle_line0, tle_line1, tle_line2)

    contacts = []
    
    if duration is None:
        # get passes for next 24 hrs
        duration = 24
        end_time = ephem.date(ground_station.date+5*365)
    if num_passes is None:
        # num_passes > max passes possible in duration.
        # duration is in hours, so 4 per hour is large
        # enough for duration to break out of loop.
        num_passes = 4 * int(duration)
        end_time = ephem.date(ground_station.date+duration*ephem.hour)
    if num_passes is not None and duration is not None:
        end_time = ephem.date(ground_station.date+duration*ephem.hour)
    
    try:
        for i in range(num_passes):
            if ground_station.date > end_time:
                break
            sat.compute(ground_station)  # compute all body attributes for sat
            # next pass command yields array with [0]=rise time,
            # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
            # [4]=set time, [5]=set azimuth
            info = ground_station.next_pass(sat)
            rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
            deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
            pass_duration = timedelta(days = set_time-rise_time)  # fraction of a day

            pass_data = {
                'start' : rise_time.datetime().ctime(),
                'end' : set_time.datetime().ctime(),
                'duration' : str(pass_duration),
                'rise_az' : (rise_az*deg_per_rad),
                'set_az' : (set_az*deg_per_rad)
            }

            if set_time > rise_time:  # only update if set time > rise time
                ground_station.date = set_time  # new obs time = prev set time
                contacts.append(pass_data)

            # increase by 1 min and look for next pass
            ground_station.date = ground_station.date + ephem.minute
    except ValueError:
        # No (more) visible passes
        pass
    return contacts

In [49]:
from itertools import islice
data = []
with open('amateur.txt') as f:
    while True:
        #an iterator that returns the next N lines and stops
        tripleline = islice(f, 3)
        #loop over these N lines, removing trailing spaces and \n
        tle = [x.rstrip() for x in tripleline]

        #only accept complete data
        #the end of the file *should* have len(tle)==0 but
        #this also handles extra junk at the end
        if len(tle) == 3:
            data.append(tle)
        else:
            break

vu = ['Valparaiso University', '41.4639', '-87.0439', 245.089]
start_time = '2017/6/14 15:00:00'
num_passes = 5
duration = 24
vu_passes = {}
new_vu_passes = {}

for tle in data:
    # use NORAD ID as key for each satellite
    # value is list of passes, where each pass is a dictionary of data
    noradID = tle[2][2:7]
    vu_passes[noradID] = get_passes(vu, tle, start_time, num_passes=num_passes, duration=duration)
    new_vu_passes[noradID] = new_get_passes(vu, tle, start_time, num_passes=num_passes, duration=duration)

In [50]:
vu_passes == new_vu_passes

True

In [46]:
import pprint
pprint.pprint(new_vu_passes)

{'07530': [{'duration': '0:07:21.011088',
            'end': 'Wed Jun 14 16:31:43 2017',
            'rise_az': 40.80047310143508,
            'set_az': 0.6288940434188611,
            'start': 'Wed Jun 14 16:24:22 2017'},
           {'duration': '0:17:51.548457',
            'end': 'Wed Jun 14 18:25:59 2017',
            'rise_az': 100.82271045193018,
            'set_az': 352.9000830171978,
            'start': 'Wed Jun 14 18:08:07 2017'},
           {'duration': '0:22:22.365498',
            'end': 'Wed Jun 14 20:19:32 2017',
            'rise_az': 150.11796837128557,
            'set_az': 345.3929401381667,
            'start': 'Wed Jun 14 19:57:10 2017'},
           {'duration': '0:20:35.293520',
            'end': 'Wed Jun 14 22:12:01 2017',
            'rise_az': 198.79873466158242,
            'set_az': 333.31126440214126,
            'start': 'Wed Jun 14 21:51:25 2017'},
           {'duration': '0:19:46.384559',
            'end': 'Thu Jun 15 08:18:40 2017',
            'rise_

,
            'end': 'Thu Jun 15 06:43:32 2017',
            'rise_az': 114.49960418936877,
            'set_az': 6.058324006061596,
            'start': 'Thu Jun 15 06:31:59 2017'},
           {'duration': '0:14:23.532478',
            'end': 'Thu Jun 15 08:22:06 2017',
            'rise_az': 168.89074330506563,
            'set_az': 345.6108778142035,
            'start': 'Thu Jun 15 08:07:42 2017'}],
 '35935': [{'duration': '0:08:30.324306',
            'end': 'Wed Jun 14 19:15:28 2017',
            'rise_az': 47.59590685385623,
            'set_az': 121.72789093515571,
            'start': 'Wed Jun 14 19:06:58 2017'},
           {'duration': '0:14:21.887295',
            'end': 'Wed Jun 14 20:57:08 2017',
            'rise_az': 16.70822048354946,
            'set_az': 185.21126176075765,
            'start': 'Wed Jun 14 20:42:46 2017'},
           {'duration': '0:12:15.730098',
            'end': 'Wed Jun 14 22:33:26 2017',
            'rise_az': 356.61677143519523,
            'se

: 1.7610282073129164,
            'start': 'Wed Jun 14 22:29:19 2017'},
           {'duration': '0:11:17.768853',
            'end': 'Thu Jun 15 00:14:17 2017',
            'rise_az': 194.23340889958922,
            'set_az': 335.43889564935694,
            'start': 'Thu Jun 15 00:03:00 2017'},
           {'duration': '0:11:24.564692',
            'end': 'Thu Jun 15 11:44:12 2017',
            'rise_az': 22.458187915568015,
            'set_az': 170.05332346407184,
            'start': 'Thu Jun 15 11:32:48 2017'},
           {'duration': '0:10:39.699629',
            'end': 'Thu Jun 15 13:17:33 2017',
            'rise_az': 356.1286861171061,
            'set_az': 228.19010416965781,
            'start': 'Thu Jun 15 13:06:53 2017'},
           {'duration': '0:10:15.719801',
            'end': 'Thu Jun 15 22:22:50 2017',
            'rise_az': 125.28728009656777,
            'set_az': 6.857923580024411,
            'start': 'Thu Jun 15 22:12:35 2017'}],
 '40901': [{'duration': '0:10:50.

'end': 'Thu Jun 15 14:58:25 2017',
            'rise_az': 29.40746826394911,
            'set_az': 156.86043059159869,
            'start': 'Thu Jun 15 14:47:56 2017'}]}
